In [1]:
import pandas as pd
import numpy as np
import datetime

# Load 
## Covid-19 data

In [2]:
OxCGRT = pd.read_csv("https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv")
OxCGRT.loc[:,'Date'] = pd.to_datetime(OxCGRT.Date, format='%Y%m%d')

C:\Users\37125\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Region data

In [3]:
regions = pd.read_csv("https://datahub.io/JohnSnowLabs/country-and-continent-codes-list/r/country-and-continent-codes-list-csv.csv")

Merge continent names to the main data set (merge on country code)

In [4]:
regions = regions[["Three_Letter_Country_Code","Continent_Name"]]
data = pd.merge(OxCGRT, regions, how='left', left_on='CountryCode', right_on='Three_Letter_Country_Code')
data = data.drop(columns=['Three_Letter_Country_Code'])

### Calculate summary by continents and globally

Get world and continent data by dates:
- Total Confirmed Cases
- Total Confirmed Deaths
- Average Stringency Index
- Average Government Response Index
- Average Containment Health Index
- Average Economic Support Index

!NB results may seem different from other sources because those sources are based on John Hopkins data, but here Oxford data is used

In [5]:
# By continents
data_continent = pd.pivot_table(data[(data.Jurisdiction=="NAT_TOTAL")],
               values=["ConfirmedCases","ConfirmedDeaths","StringencyIndexForDisplay","GovernmentResponseIndexForDisplay",
                       "ContainmentHealthIndexForDisplay","EconomicSupportIndexForDisplay"],
               index=["Date","Continent_Name"], 
               aggfunc={"ConfirmedCases":np.sum,
                        "ConfirmedDeaths":np.sum,
                        "StringencyIndexForDisplay":np.mean,
                        "GovernmentResponseIndexForDisplay":np.mean,
                        "ContainmentHealthIndexForDisplay":np.mean,
                        "EconomicSupportIndexForDisplay":np.mean}
              ).reset_index()
# World
data_world = pd.pivot_table(data[(data.Jurisdiction=="NAT_TOTAL")],
               values=["ConfirmedCases","ConfirmedDeaths","StringencyIndexForDisplay","GovernmentResponseIndexForDisplay",
                       "ContainmentHealthIndexForDisplay","EconomicSupportIndexForDisplay"],
               index=["Date"], 
               aggfunc={"ConfirmedCases":np.sum,
                        "ConfirmedDeaths":np.sum,
                        "StringencyIndexForDisplay":np.mean,
                        "GovernmentResponseIndexForDisplay":np.mean,
                        "ContainmentHealthIndexForDisplay":np.mean,
                        "EconomicSupportIndexForDisplay":np.mean}
              ).reset_index()
data_world.loc[:,"Continent_Name"] = "World"

# Concatenate
data_summary = pd.concat([data_continent,data_world]).reset_index(drop=True)

# Preview
data_summary[data_summary.Date=="2021-03-28"]

,Date,Continent_Name,ConfirmedCases,ConfirmedDeaths,ContainmentHealthIndexForDisplay,EconomicSupportIndexForDisplay,GovernmentResponseIndexForDisplay,StringencyIndexForDisplay
2712,2021-03-28,Africa,4172344.0,111716.0,48.709400,21.250000,45.277600,48.111200
2713,2021-03-28,Asia,32309854.0,517063.0,58.714286,49.744898,57.593061,56.500612
2714,2021-03-28,Europe,42858387.0,937960.0,65.744348,60.326087,65.134783,63.950000
2715,2021-03-28,North America,35007520.0,803842.0,58.835217,41.304348,56.644783,58.191739
2716,2021-03-28,Oceania,44841.0,1116.0,38.867778,29.166667,37.655556,37.087778
2717,2021-03-28,South America,20809231.0,540704.0,65.679167,59.375000,64.372500,68.942500
3176,2021-03-28,World,135286349.0,2914212.0,57.305263,43.486842,55.561895,56.196526


### Clean covid-19 data

Remove extra columns and rows (countries with atypical data)

In [6]:
columns_of_interest = ['CountryName', 'CountryCode', 'Continent_Name', 'Date', 'ConfirmedCases', 'ConfirmedDeaths',
                       'C1_School closing','C2_Workplace closing','C3_Cancel public events',
                       'C4_Restrictions on gatherings', 'C5_Close public transport','C6_Stay at home requirements',
                       'C7_Restrictions on internal movement','C8_International travel controls',
                       'E1_Income support','E2_Debt/contract relief', 'E3_Fiscal measures','E4_International support', 
                       'H1_Public information campaigns','H2_Testing policy', 'H3_Contact tracing','H4_Emergency investment in healthcare', 
                       'H5_Investment in vaccines','H6_Facial Coverings', 'H7_Vaccination policy', 'H8_Protection of elderly people', 
                       'M1_Wildcard',
                       'StringencyIndex','StringencyIndexForDisplay','StringencyLegacyIndex','StringencyLegacyIndexForDisplay', 
                       'GovernmentResponseIndex','GovernmentResponseIndexForDisplay', 
                       'ContainmentHealthIndex','ContainmentHealthIndexForDisplay', 
                       'EconomicSupportIndex','EconomicSupportIndexForDisplay']
excluded_countries = ['United Kingdom','Canada','Brazil','United States', # because have npi data only by regions
                      'Kosovo', 'Taiwan','Eritrea'] # No population data

data = data[~data.CountryName.isin(excluded_countries)][columns_of_interest]

In [7]:
data.head(3)

,CountryName,CountryCode,Continent_Name,Date,ConfirmedCases,ConfirmedDeaths,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,Aruba,ABW,North America,2020-01-01,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,ABW,North America,2020-01-02,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,ABW,North America,2020-01-03,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Population data

In [8]:
# Source: https://data.worldbank.org/indicator/SP.POP.TOTL
population = pd.read_csv("data/population.csv",skiprows=4)
population = population[["Country Code","2019"]]

# Calculate continent summary
population_summary = pd.merge(population, regions, how='left', left_on='Country Code', right_on='Three_Letter_Country_Code')
population_summary = population_summary.drop(columns=['Three_Letter_Country_Code'])
population_summary = population_summary.groupby("Continent_Name").sum().reset_index()
population_summary = pd.concat([population_summary,
                                pd.DataFrame(data={"Continent_Name":"World",
                                                   "2019":[population_summary["2019"].sum()]})])

# Get country names from 'data' table to ensure identical naming
population = pd.merge(population, data[['CountryCode','CountryName']].drop_duplicates(),
                      left_on='Country Code', right_on='CountryCode')
population = population[["CountryName","2019"]].set_index("CountryName")
population.index.name = None

In [9]:
population.head(3)

,2019
Aruba,106314.0
Afghanistan,38041754.0
Angola,31825295.0


In [10]:
population_summary

,Continent_Name,2019
0,Africa,1.302819e+09
1,Asia,4.686780e+09
2,Europe,8.643194e+08
3,North America,5.853165e+08
4,Oceania,4.239802e+07
5,South America,4.270633e+08
0,World,7.908696e+09


## Tests data

In [11]:
tests = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/testing/covid-testing-all-observations.csv")

In [12]:
tests.loc[:,~tests.columns.isin(["Entity","Source URL","Source label","Notes"])].head(3)

,ISO code,Date,Daily change in cumulative total,Cumulative total,Cumulative total per thousand,Daily change in cumulative total per thousand,7-day smoothed daily change,7-day smoothed daily change per thousand,Short-term positive rate,Short-term tests per case
0,ALB,2020-02-25,8.0,8.0,0.003,0.003,NaN,NaN,NaN,NaN
1,ALB,2020-02-26,5.0,13.0,0.005,0.002,NaN,NaN,NaN,NaN
2,ALB,2020-02-27,4.0,17.0,0.006,0.001,NaN,NaN,NaN,NaN


In [13]:
tests.loc[:,'Date'] = pd.to_datetime(tests.Date, format='%Y-%m-%d')
tests = tests.loc[:,~tests.columns.isin(["Entity","Source URL","Source label","Notes"])]
data = pd.merge(data, tests, how='left', left_on=["CountryCode","Date"], right_on=["ISO code","Date"])
data = data.drop(columns=['ISO code'])
data = data.rename(columns={"Cumulative total":"Tests_total"})

Continent summary

In [14]:
tests_summary = pd.merge(tests, regions, how='left', left_on='ISO code', right_on='Three_Letter_Country_Code')
tests_summary = tests_summary.drop(columns=['Three_Letter_Country_Code'])
tests_summary = tests_summary.groupby(["Date","Continent_Name"])["Cumulative total"].sum().reset_index()
tests_world = tests_summary.groupby("Date")["Cumulative total"].sum().reset_index()
tests_world.loc[:,"Continent_Name"] = "World"
tests_summary = pd.concat([tests_summary, tests_world]).reset_index(drop=True)
tests_summary = tests_summary.rename(columns={"Cumulative total":"Tests_total"})

In [15]:
tests_summary[tests_summary.Date=='2021-03-25']

,Date,Continent_Name,Tests_total
2593,2021-03-25,Africa,19594428.0
2594,2021-03-25,Asia,527874120.0
2595,2021-03-25,Europe,300404523.0
2596,2021-03-25,North America,26977663.0
2597,2021-03-25,Oceania,17239146.0
2598,2021-03-25,South America,25551790.0
3052,2021-03-25,World,917641670.0


In [16]:
for continent in tests_summary.Continent_Name.unique():
    population_100K = population_summary[population_summary.Continent_Name == continent]["2019"].values / 100000
    
    tests_summary.loc[tests_summary.Continent_Name==continent, "Tests_total_100K"]\
        = tests_summary[tests_summary.Continent_Name==continent].Tests_total \
        / population_100K
tests_summary[tests_summary.Date=='2021-03-25']

,Date,Continent_Name,Tests_total,Tests_total_100K
2593,2021-03-25,Africa,19594428.0,1504.001850
2594,2021-03-25,Asia,527874120.0,11263.044927
2595,2021-03-25,Europe,300404523.0,34756.195255
2596,2021-03-25,North America,26977663.0,4609.072635
2597,2021-03-25,Oceania,17239146.0,40660.261006
2598,2021-03-25,South America,25551790.0,5983.139318
3052,2021-03-25,World,917641670.0,11602.944695


Update `data_summary`

In [17]:
#data_summary = pd.merge(data_summary, tests_summary)

## Vaccination data

In [18]:
vaccine = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv")

In [19]:
vaccine.head(2)

,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0


In [20]:
vaccine.loc[:,'date'] = pd.to_datetime(vaccine.date, format='%Y-%m-%d')
vaccine = vaccine.loc[:,vaccine.columns != "location"]
data = pd.merge(data, vaccine, how='left', left_on=["CountryCode","Date"], right_on=["iso_code","date"])
data = data.drop(columns=["iso_code","date"])

Continent summary

In [21]:
vaccine_summary = pd.merge(vaccine, regions, how='left', left_on='iso_code', right_on='Three_Letter_Country_Code')
vaccine_summary = vaccine_summary.drop(columns=['Three_Letter_Country_Code'])
vaccine_summary = vaccine_summary.groupby(["date","Continent_Name"])[["total_vaccinations","people_vaccinated"]].sum().reset_index()
vaccine_world = vaccine_summary.groupby("date")[["total_vaccinations","people_vaccinated"]].sum().reset_index()
vaccine_world.loc[:,"Continent_Name"] = "World"
vaccine_summary = pd.concat([vaccine_summary, vaccine_world]).reset_index(drop=True)

vaccine_summary[vaccine_summary.date=='2021-03-25']

,date,Continent_Name,total_vaccinations,people_vaccinated
507,2021-03-25,Africa,7965792.0,4894675.0
508,2021-03-25,Asia,201785145.0,75281344.0
509,2021-03-25,Europe,104544754.0,78189092.0
510,2021-03-25,North America,144684934.0,96918778.0
511,2021-03-25,Oceania,408410.0,0.0
512,2021-03-25,South America,32329125.0,24122194.0
633,2021-03-25,World,491718160.0,279406083.0


In [22]:
for continent in vaccine_summary.Continent_Name.unique():
    population_100K = population_summary[population_summary.Continent_Name == continent]["2019"].values / 100000
    
    vaccine_summary.loc[vaccine_summary.Continent_Name==continent, "total_vaccination_100K"]\
        = vaccine_summary[vaccine_summary.Continent_Name==continent].total_vaccinations \
        / population_100K
    
    vaccine_summary.loc[vaccine_summary.Continent_Name==continent, "people_vaccinated_100K"]\
        = vaccine_summary[vaccine_summary.Continent_Name==continent].people_vaccinated \
        / population_100K  

In [23]:
continent = "Asia"
population_summary[population_summary.Continent_Name == continent]["2019"].values

array([4.68677985e+09])

In [24]:
vaccine_summary[vaccine_summary.date=='2021-03-25']

,date,Continent_Name,total_vaccinations,people_vaccinated,total_vaccination_100K,people_vaccinated_100K
507,2021-03-25,Africa,7965792.0,4894675.0,611.427183,375.698656
508,2021-03-25,Asia,201785145.0,75281344.0,4305.411210,1606.248777
509,2021-03-25,Europe,104544754.0,78189092.0,12095.616426,9046.319680
510,2021-03-25,North America,144684934.0,96918778.0,24719.093349,16558.353758
511,2021-03-25,Oceania,408410.0,0.0,963.276093,0.000000
512,2021-03-25,South America,32329125.0,24122194.0,7570.102090,5648.388913
633,2021-03-25,World,491718160.0,279406083.0,6217.436286,3532.896810


# Calculate
## Covid cases

- Daily cases
- Daily cases per 100K population
- 7-day-average
- 7-day-average per 100K population
- 14-day-average
- 14-day-average per 100K population

In [25]:
for country in data.CountryName.unique():
    population_100K = population.loc[country].values / 100000
    data.loc[(data.CountryName==country),"ConfirmedCases_100K"] = data[(data.CountryName==country)]["ConfirmedCases"] / population_100K
    data.loc[(data.CountryName==country),"ConfirmedDeaths_100K"] = data[(data.CountryName==country)]["ConfirmedDeaths"] / population_100K
    data.loc[(data.CountryName==country),"DailyCases"] = data[(data.CountryName==country)]["ConfirmedCases"].diff()
    data.loc[(data.CountryName==country),"DailyCases_100K"] = data[(data.CountryName==country)]["DailyCases"] / population_100K
    data.loc[(data.CountryName==country),"Average7"] = data[(data.CountryName==country)]["ConfirmedCases"].diff().rolling(7).mean()
    data.loc[(data.CountryName==country),"Average7_100K"] = data[(data.CountryName==country)]["Average7"] / population_100K
    data.loc[(data.CountryName==country),"Average14"] = data[(data.CountryName==country)]["ConfirmedCases"].diff().rolling(14).mean()
    data.loc[(data.CountryName==country),"Average14_100K"] = data[(data.CountryName==country)]["Average14"] / population_100K
    data.loc[(data.CountryName==country),"total_vaccination_100K"] = data[(data.CountryName==country)]["total_vaccinations"] / population_100K
    data.loc[(data.CountryName==country),"people_vaccinated_100K"] = data[(data.CountryName==country)]["people_vaccinated"] / population_100K
    data.loc[(data.CountryName==country),"Tests_total_100K"] = data[(data.CountryName==country)]["Tests_total"] / population_100K

## Relative to world/continent

- Normalized total cases
- Normalized 14-day average cases
- Normalized total deaths 
- Normalized 14-day average death (?)
- Normalized total tests
- Normalized total vaccines
- Normalized total people vaccinated
- Stringency Index
- Economic Support Index

In [26]:
# for country in data.CountryName.unique():
#     continent = data.loc[(data.CountryName==country),"Continent_Name"].max()
#     world = "World"
    

In [27]:
for continent in data_summary.Continent_Name.unique():
    population_100K = population_summary[population_summary.Continent_Name == continent]["2019"].values / 100000
    
    data_summary.loc[data_summary.Continent_Name==continent, "ConfirmedCases_100K"]\
        = data_summary[data_summary.Continent_Name==continent].ConfirmedCases / population_100K
 
    data_summary.loc[data_summary.Continent_Name==continent, "ConfirmedDeaths_100K"]\
        = data_summary[data_summary.Continent_Name==continent].ConfirmedDeaths / population_100K
    
    data_summary.loc[data_summary.Continent_Name==continent,"Average14_100K"]\
        = data_summary[data_summary.Continent_Name==continent]["ConfirmedCases"].diff().rolling(14).mean() / population_100K

,Date,Continent_Name,ConfirmedCases,ConfirmedDeaths,ContainmentHealthIndexForDisplay,EconomicSupportIndexForDisplay,GovernmentResponseIndexForDisplay,StringencyIndexForDisplay,ConfirmedCases_100K,ConfirmedDeaths_100K,Average14_100K
2694,2021-03-25,Africa,4135438.0,110996.0,48.614200,21.250000,45.194400,47.963200,317.422198,8.519677,0.769737
2695,2021-03-25,Asia,31813320.0,512653.0,58.714286,49.744898,57.593061,56.500612,678.788444,10.938278,2.668437
2696,2021-03-25,Europe,42147118.0,928058.0,65.614565,60.326087,65.021522,63.748478,4876.336242,107.374432,24.504377
2697,2021-03-25,North America,34785209.0,799800.0,58.731739,41.304348,56.553913,58.030870,5942.974190,136.644019,11.838906
2698,2021-03-25,Oceania,43709.0,1110.0,38.603333,29.166667,37.424444,37.087778,103.092076,2.618047,0.425559
2699,2021-03-25,South America,20465329.0,529839.0,64.240833,59.375000,63.113333,66.705000,4792.107112,124.065694,25.041422
3173,2021-03-25,World,133474295.0,2884267.0,57.132895,43.486842,55.411105,55.948000,1687.690210,36.469563,6.625395


In [28]:
data_summary = pd.merge(data_summary, vaccine_summary, how='left',
                        left_on=["Date","Continent_Name"], right_on=["date","Continent_Name"]
                       ).drop(columns=['date'])
data_summary = pd.merge(data_summary, tests_summary, how='left', on=["Date","Continent_Name"])
data_summary[data_summary.Date=='2021-03-25']

,Date,Continent_Name,ConfirmedCases,ConfirmedDeaths,ContainmentHealthIndexForDisplay,EconomicSupportIndexForDisplay,GovernmentResponseIndexForDisplay,StringencyIndexForDisplay,ConfirmedCases_100K,ConfirmedDeaths_100K,Average14_100K,total_vaccinations,people_vaccinated,total_vaccination_100K,people_vaccinated_100K,Tests_total,Tests_total_100K
2694,2021-03-25,Africa,4135438.0,110996.0,48.614200,21.250000,45.194400,47.963200,317.422198,8.519677,0.769737,7965792.0,4894675.0,611.427183,375.698656,19594428.0,1504.001850
2695,2021-03-25,Asia,31813320.0,512653.0,58.714286,49.744898,57.593061,56.500612,678.788444,10.938278,2.668437,201785145.0,75281344.0,4305.411210,1606.248777,527874120.0,11263.044927
2696,2021-03-25,Europe,42147118.0,928058.0,65.614565,60.326087,65.021522,63.748478,4876.336242,107.374432,24.504377,104544754.0,78189092.0,12095.616426,9046.319680,300404523.0,34756.195255
2697,2021-03-25,North America,34785209.0,799800.0,58.731739,41.304348,56.553913,58.030870,5942.974190,136.644019,11.838906,144684934.0,96918778.0,24719.093349,16558.353758,26977663.0,4609.072635
2698,2021-03-25,Oceania,43709.0,1110.0,38.603333,29.166667,37.424444,37.087778,103.092076,2.618047,0.425559,408410.0,0.0,963.276093,0.000000,17239146.0,40660.261006
2699,2021-03-25,South America,20465329.0,529839.0,64.240833,59.375000,63.113333,66.705000,4792.107112,124.065694,25.041422,32329125.0,24122194.0,7570.102090,5648.388913,25551790.0,5983.139318
3173,2021-03-25,World,133474295.0,2884267.0,57.132895,43.486842,55.411105,55.948000,1687.690210,36.469563,6.625395,491718160.0,279406083.0,6217.436286,3532.896810,917641670.0,11602.944695


In [37]:
columns = ["Continent_Name","ConfirmedCases_100K","Average14_100K","ConfirmedDeaths_100K",
           "Tests_total_100K","total_vaccination_100K","people_vaccinated_100K",
           "StringencyIndexForDisplay", "EconomicSupportIndexForDisplay"]
data_summary[(data_summary.Date=='2021-03-20') & (data_summary.Continent_Name.isin(['Europe','World']))][columns]

,Continent_Name,ConfirmedCases_100K,Average14_100K,ConfirmedDeaths_100K,Tests_total_100K,total_vaccination_100K,people_vaccinated_100K,StringencyIndexForDisplay,EconomicSupportIndexForDisplay
2666,Europe,4745.547200,21.981111,105.329586,43179.827851,12067.539189,9061.619696,64.942500,58.437500
3168,World,1652.102892,6.015983,35.865051,15666.256518,5664.184703,3294.197471,55.871146,42.307692


In [35]:
data[(data.Date=='2021-03-20') & (data.CountryName=='Lithuania')][['CountryName']+columns]

,CountryName,Continent_Name,ConfirmedCases_100K,Average14_100K,ConfirmedDeaths_100K,Tests_total_100K,total_vaccination_100K,people_vaccinated_100K
47084,Lithuania,Europe,7486.963748,17.559546,124.29831,81925.181316,14401.272551,9916.845005


In [39]:
pd.concat([data_summary[(data_summary.Date=='2021-03-20') & (data_summary.Continent_Name.isin(['Europe','World']))][columns],
           data[(data.Date=='2021-03-20') & (data.CountryName=='Lithuania')][['CountryName']+columns]]
         ).to_csv("LTvsWorld_20210320-1.csv")

# Save to the file

In [40]:
data.to_csv("data/OxCGRT_latest.csv")

In [41]:
data_summary.to_csv("data/summary.csv")